<a href="https://colab.research.google.com/github/totalleecher/Codemaster/blob/master/codemaster%40bssg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent to Gdrive (Created by Codemaster@BSSG)

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.

### Install libtorrent and Initialize Session

In [0]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libtorrent-rasterbar9
Suggested packages:
  libtorrent-rasterbar-dbg
The following NEW packages will be installed:
  libtorrent-rasterbar9 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,569 kB of archives.
After this operation, 5,718 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtorrent-rasterbar9 amd64 1.1.5-1build1 [1,258 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-libtorrent amd64 1.1.5-1build1 [310 kB]
Fetched 1,569 kB in 1s (1,777 kB/s)
Selecting previously unselected package libtorrent-rasterbar9.
(Reading database ... 134443 files and directories currently insta

### Mount Google Drive
To stream files we need to mount Google Drive.Better run with unlimited googledrive or Team Drive which you can purchase only with 1000 mmk :3

In [0]:
from google.colab import drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [0]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [0]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:03c1ed36120068c02b7e24528a11f40e8d84fb33&dn=%5BJAV%5D%20%5BUncensored%5D%20MIDE-193%20Uncensored%20Leaked%20%E7%A5%9E%E5%92%B2%E8%A9%A9%E7%B9%94%E3%81%8C%E3%81%82%E3%81%AA%E3%81%9F%E3%81%AE%E3%81%8A%E5%AB%81%E3%81%95%E3%82%93%20%E3%83%A2%E3%82%B6%E3%82%A4%E3%82%AF%E7%A0%B4%E5%A3%8A%E7%89%88%20%5B1080p%5D&tr=http%3A%2F%2Fsukebei.tracker.wf%3A8888%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce
Enter Magnet Link Or Type Exit: exit


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [0]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


NameError: ignored

Hope You Enjoy. 

Sincerly,
BSSG Admin Team